# Predictive Maintenance of a Water Pump

**Aim:** Given the sensor data, predict water pump failure in advance to guide timely maintenance scheduling. 

**Dataset:** [pump_sensor_data](https://www.kaggle.com/datasets/nphantawee/pump-sensor-data/metadata)

### Setup
Import the necessary libraries.

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras import models

### Get Data

Load the pump sensor data.

In [7]:
# parse timestamp column as datetime object upon reading the data
df_raw_data = pd.read_csv('data/sensor.csv', parse_dates=['timestamp'], index_col=0)

## Data Exploration

In [10]:
# initial look at the features
df_raw_data.head()

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL


In [9]:
# determine the size of the dataset
print('The dataset has the following shape: {}'.format(df_raw_data.shape))

The dataset has the following shape: (220320, 54)


To sum up, there are 52 sensors with a measurement made every minute for 220320 minutes (153 days). The `machine_status` column is the target feature that will be used to predict.. And will be explored next. 

In [12]:
# investigate the number of features for each target feature
df_raw_data.groupby('machine_status')['machine_status'].count()

machine_status
BROKEN             7
NORMAL        205836
RECOVERING     14477
Name: machine_status, dtype: int64